In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras_tuner as kt
import matplotlib.pyplot as plt
import seaborn as sns

### 1. data preprorcessing: prepare data for training & test sets

#### import dataset and data cleaning and selection 

The training dataset includes wind speeds of various altitude increments, in 10 minutes intervals, for a little more than 12 month. It includes horizontal wind speed, vertical wind speed, mean values, max and min values in the increments, standard deviations and other columns. 

There are certain unrealistic values for wind speeds suh as 9999 or 9998 or N/A, this is likely due to processing error in the technical instrumet.

Thus, for data cleaning, we will filter out the values above 50 as it is nearly impossible for wind speed in the given altitudes to exceed 50. And for data selection, we shall only use the columns that record the horizontal wind speeds as the HAWTs (Horizontal-Axis Wind Turbines) use the lift force of the horizontal winds to generate energy, and disregard the other data for now.  

In [2]:
file_path = "../Data/E05_Hudson_North_10_min_avg_20190812_20210919.csv"

data = pd.read_csv(file_path)

data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')

sorted_data = data[data['timestamp'] <= pd.Timestamp('2021-08-12')]

sorted_data.head()

C:\Users\Jiarong Xie\AppData\Local\Temp\ipykernel_33424\1948449413.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,62,63,64,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


,timestamp,lidar_lidar18m_Z10_HorizWS,lidar_lidar18m_Z10_StdDevWS,lidar_lidar18m_Z10_MaxWS,lidar_lidar18m_Z10_MinWs,lidar_lidar18m_WD_alg_03,lidar_lidar18m_Z10_VertWs,lidar_lidar18m_Z10_InfoFlag,lidar_lidar18m_Z10_StatFlag,lidar_lidar18m_Z10_Packets,...,AHRS_AHRSroll_Max,AHRS_AHRSroll_Min,AHRS_AHRSpitch_Max,AHRS_AHRSpitch_Min,AHRS_AHRSyaw_Max,AHRS_AHRSyaw_Min,buoy_status_CR6S_batt_Avg,buoy_status_GPSlat,buoy_status_GPSlong,FLS200_S/N
0,2019-08-12 00:00:00,7.2923,0.5159,8.2792,6.0537,251.5426,0.1971,4.0,0.0,36.0,...,7.644,-6.274,6.088,-7.194,6.121,-6.098,13.03,39.9695,-72.716,E05
1,2019-08-12 00:10:00,7.8613,0.6321,9.1869,6.1966,245.2114,0.1425,4.0,0.0,37.0,...,7.205,-6.716,6.86,-8.25,6.024,-5.765,13.03,39.9694,-72.716,E05
2,2019-08-12 00:20:00,7.7594,0.635,9.0702,6.1366,246.7355,0.1568,4.0,0.0,37.0,...,6.416,-7.058,5.03,-7.563,5.344,-6.38,13.03,39.9695,-72.716,E05
3,2019-08-12 00:30:00,7.4891,0.4983,8.6377,6.5824,247.0218,0.0493,4.0,0.0,38.0,...,7.767,-7.505,5.555,-7.663,7.64,-8.3,13.03,39.9695,-72.716,E05
4,2019-08-12 00:40:00,7.8829,0.6361,9.4056,6.4107,244.106,0.0897,4.0,0.0,37.0,...,6.627,-6.468,7.694,-9.47,6.511,-7.137,13.02,39.9695,-72.716,E05


In [3]:
# Filtering columns with 'HorizWS' in their names
horiz_ws_columns = [col for col in data.columns if 'lidar_lidar138m_Z10_HorizWS' in col]
horiz_ws_data = data[horiz_ws_columns]

# Converting wind speed columns to numeric, setting errors='coerce' to turn non-numeric values into NaN
numeric_horiz_ws_data = horiz_ws_data.apply(pd.to_numeric, errors='coerce')

# Filtering out all values above 50
cleaned_numeric_horiz_ws_data = numeric_horiz_ws_data[numeric_horiz_ws_data <= 50].dropna()

# Displaying the first few rows of the cleaned numeric data
cleaned_numeric_horiz_ws_data.head()

,lidar_lidar138m_Z10_HorizWS
0,7.8635
1,8.1698
2,8.1291
3,7.9652
4,8.0443


In [4]:
len(cleaned_numeric_horiz_ws_data)

100711

In [5]:
points_in_two_years = 6 * 24 * 365 * 2

# Generate new index for interpolation
new_index = np.linspace(0, len(cleaned_numeric_horiz_ws_data) - 1, points_in_two_years)

# Interpolating the DataFrame
interpolated_data = np.interp(new_index, np.arange(len(cleaned_numeric_horiz_ws_data)), 
                              cleaned_numeric_horiz_ws_data.iloc[:, 0])

# Converting the interpolated array back to a DataFrame
cleaned_numeric_horiz_ws_data = pd.DataFrame(interpolated_data, columns=cleaned_numeric_horiz_ws_data.columns)

In [6]:
len(cleaned_numeric_horiz_ws_data)

105120

#### Data normalization

The data is normalized with MinMax (imported from the scikit-learn library) for MinMax scaling's ability to transform data into a bounded range while preserving temporal relationships. It is suitable for TimeGAN models like this one, which are designed to generate realistic time-series data.

In [7]:
# Initializing the MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fitting the scaler to the cleaned data and transforming it
normalized_data = scaler.fit_transform(cleaned_numeric_horiz_ws_data)

# Converting the normalized data back to a DataFrame for better readability
normalized_df = pd.DataFrame(normalized_data, columns=cleaned_numeric_horiz_ws_data.columns)

# Displaying the first few rows of the normalized data
normalized_df.head()

,lidar_lidar138m_Z10_HorizWS
0,0.246939
1,0.256687
2,0.255875
3,0.251002
4,0.252504


#### Reshaping the data

LSTM model is chosen to focus on seasonal variations so the previos

In [8]:
points_in_month = int(len(cleaned_numeric_horiz_ws_data) / (2 * 12))
points_in_month

4380

In [9]:
ratio = [1, 11]
forecast_ratio = int(ratio[1] / ratio[0])

In [10]:
input_sequence_length = points_in_month * ratio[0]
output_sequence_length = points_in_month * ratio[1]

In [11]:
def create_sequences(data, history_length, forecast_ratio):
    X, Y = [], []
    forecast_length = history_length * forecast_ratio
    total_sequence_length = history_length + forecast_length

    # Check if the total sequence length is longer than the data length
    if total_sequence_length > len(data):
        print("Total sequence length is greater than the data length. Adjusting the lengths.")
        # Adjust the lengths to fit the data
        forecast_length = len(data) - history_length
        total_sequence_length = len(data)

    for i in range(0, len(data) - total_sequence_length + 1, total_sequence_length):
        X.append(data[i:(i + history_length)])
        Y.append(data[(i + history_length):(i + total_sequence_length)])
    return np.array(X), np.array(Y)

# Assuming input_sequence_length is defined
X, Y = create_sequences(normalized_df, input_sequence_length, forecast_ratio)

# Splitting the data in half for training and testing
train_size = len(X) // 2
X_train, Y_train = X[:train_size], Y[:train_size]
X_test, Y_test = X[train_size:], Y[train_size:]

# Print shapes to verify
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (1, 4380, 1)
Y_train shape: (1, 48180, 1)
X_test shape: (1, 4380, 1)
Y_test shape: (1, 48180, 1)


#### Defining the LSTM model

In [12]:
num_epochs = 50
minibatch_size = 64
n_trials = 10
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 20, verbose = 1, restore_best_weights = True)

In [13]:
def build_model(hp):

    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    hp_learning_rate = hp.Float('learning_rate', min_value = 1e-4, max_value = 1e-2, sampling = 'LOG')
    hp_loss = hp.Choice('loss', values=['mse','mae'])

    model = keras.Sequential()
    
    model.add(LSTM(
        hp.Int('neurons', min_value = 32, max_value = 512, step = 32),
        input_shape = (X_train.shape[1], X_train.shape[2]),
        return_sequences = True,
        activation = hp_activation
    ))

    model.add(Dropout(rate = hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(LSTM(
        hp.Int('neurons', min_value = 32, max_value = 512, step = 32),
        return_sequences = False,
        activation = hp_activation
    ))

    model.add(Dropout(rate=hp.Float('dropout_2', min_value = 0.0, max_value = 0.5, step = 0.1)))

    model.add(Dense(1, activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))

    model.compile(loss = hp_loss, optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate))

    return model

In [14]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_loss',
    max_trials = n_trials,  # Number of trials
    executions_per_trial = 1,
    directory = 'my_dir',
    project_name = 'lstm_tuning_18m'
)

tuner.search_space_summary()

Search space summary
Default search space size: 6
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
loss (Choice)
{'default': 'mse', 'conditions': [], 'values': ['mse', 'mae'], 'ordered': False}
neurons (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}


In [15]:
tuner.search(X_train, Y_train, epochs = num_epochs, validation_data=(X_test, Y_test))

Trial 10 Complete [00h 14m 42s]
val_loss: 0.13426126539707184

Best val_loss So Far: 0.027121547609567642
Total elapsed time: 01h 09m 13s


In [16]:
best_model = tuner.get_best_models(num_models = 1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model.summary()
print(best_hyperparameters.values)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4380, 352)         498432    
                                                                 
 dropout (Dropout)           (None, 4380, 352)         0         
                                                                 
 lstm_1 (LSTM)               (None, 352)               992640    
                                                                 
 dropout_1 (Dropout)         (None, 352)               0         
                                                                 
 dense (Dense)               (None, 1)                 353       
                                                                 
Total params: 1491425 (5.69 MB)
Trainable params: 1491425 (5.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
{'activation': 'si